# Stackup export

## Set up example environment

In [1]:
%run helpers/example_setup.ipynb

db, cell, layout variables set to the default from edb design.


## Get Layers

In [2]:
lc = layout.layer_collection
layers = lc.get_layers()
for i in layers:
    if i.is_stackup_layer:
        name = i.name
        layer_type = i.type.name
        material = i.get_material()
        print(f"Layer '{name}', type '{layer_type}', material '{i.get_material()}'")

Layer 's1', type 'SIGNAL_LAYER', material 'copper'
Layer 'd1', type 'DIELECTRIC_LAYER', material 'FR4_epoxy'
Layer 's2', type 'SIGNAL_LAYER', material 'copper'
Layer 'd2', type 'DIELECTRIC_LAYER', material 'FR4_epoxy'
Layer 's3', type 'SIGNAL_LAYER', material 'copper'


## Extract layer information

In [3]:

layer_info = []
for i in layers:
    if i.is_stackup_layer:
        name = i.name
        layer_type = i.type.name
        material = i.get_material()
        thickness = i.thickness.value
        lower_elevation = i.lower_elevation.value
        full_material = i.get_fill_material()
        
        if 'SIGNAL_LAYER' in str(layer_type):
            matmodel = material_def.MaterialDef.find_by_name(db,i.get_material())
            Sigma = matmodel.get_property(material_def.MaterialProperty.CONDUCTIVITY).value
            x = (name,
            layer_type,
            material,
            Sigma,
            thickness,
            lower_elevation,
            full_material)                
        elif 'DIELECTRIC_LAYER' in str(layer_type):
            matmodel = material_def.MaterialDef.find_by_name(db,i.get_material())
            Td = matmodel.get_property(material_def.MaterialProperty.DIELECTRIC_LOSS_TANGENT).value
            Er = matmodel.get_property(material_def.MaterialProperty.PERMITTIVITY).value
            x = (name,
            layer_type,
            material,
            Er,
            Td,
            thickness,
            lower_elevation,
            full_material)
        layer_info.append(x)

## Write layer information to json file

In [4]:
import json
stackup_file = os.path.join(get_output_dir(),'Stackup_export.json')
with open(stackup_file, 'w') as f:
    json.dump(layer_info, f, indent=' ')

## Display the json

In [5]:
with open(stackup_file) as f:
    d = json.load(f)
    print(d)

[['s1', 'SIGNAL_LAYER', 'copper', 58000000.0, 0.0001, 0.0034, 'FR4_epoxy'], ['d1', 'DIELECTRIC_LAYER', 'FR4_epoxy', 4.4, 0.02, 0.0016, 0.0018000000000000002, ''], ['s2', 'SIGNAL_LAYER', 'copper', 58000000.0, 0.0001, 0.0017, 'FR4_epoxy'], ['d2', 'DIELECTRIC_LAYER', 'FR4_epoxy', 4.4, 0.02, 0.0016, 0.0001, ''], ['s3', 'SIGNAL_LAYER', 'copper', 58000000.0, 0.0001, 0.0, 'FR4_epoxy']]


## Tear down example environment


In [6]:
%run helpers/example_teardown.ipynb